In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Test2').getOrCreate()
print(spark)


In [8]:
df1 = spark.read.csv('sales_first_half.csv',header = True)
df2 = spark.read.csv('sales_second_half.csv',header = True) 
df1.show(n=5)
df2.show(n=5)

+-------+----------+----------+-------------+--------------+------------------+
|sale_id|      date|product_id|quantity_sold|price_per_unit|        total_sale|
+-------+----------+----------+-------------+--------------+------------------+
|      1|2024-04-12|      1036|            7|         21.54|            150.78|
|      2|2024-06-28|      1048|            3|         33.87|101.60999999999999|
|      3|2024-04-02|      1065|            4|         85.17|            340.68|
|      4|2024-01-15|      1035|            3|         39.44|            118.32|
|      5|2024-04-16|      1053|            1|         53.93|             53.93|
+-------+----------+----------+-------------+--------------+------------------+
only showing top 5 rows

+-------+----------+----------+-------------+--------------+-----------------+
|sale_id|      date|product_id|quantity_sold|price_per_unit|       total_sale|
+-------+----------+----------+-------------+--------------+-----------------+
|      1|2024-11-0

In [10]:
df_join = df1.join(df2, on = 'product_id' , how = 'inner')
df_join.show()

+----------+-------+----------+-------------+--------------+----------+-------+----------+-------------+--------------+------------------+
|product_id|sale_id|      date|quantity_sold|price_per_unit|total_sale|sale_id|      date|quantity_sold|price_per_unit|        total_sale|
+----------+-------+----------+-------------+--------------+----------+-------+----------+-------------+--------------+------------------+
|      1036|      1|2024-04-12|            7|         21.54|    150.78|   4900|2024-09-01|            9|         37.66|338.93999999999994|
|      1036|      1|2024-04-12|            7|         21.54|    150.78|   4897|2024-09-10|            8|         14.87|            118.96|
|      1036|      1|2024-04-12|            7|         21.54|    150.78|   4831|2024-08-31|            4|         99.83|            399.32|
|      1036|      1|2024-04-12|            7|         21.54|    150.78|   4788|2024-09-15|            7|         79.39|            555.73|
|      1036|      1|2024-04

In [34]:
df_union = df1.union(df2)
df_union.show()
df_filtered = df_union.filter((df_union['date'] > '2024-06-30') & (df_union['date'] < '2024-12-30'))
df_filtered.show()

from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
@pandas_udf(IntegerType())
def add_quantity(column):
    return column + 1
df_new = df_filtered.withColumn('new_quantity', add_quantity(df_filtered['quantity_sold'].cast('int')))
df_new.show()

+-------+----------+----------+-------------+--------------+------------------+
|sale_id|      date|product_id|quantity_sold|price_per_unit|        total_sale|
+-------+----------+----------+-------------+--------------+------------------+
|      1|2024-04-12|      1036|            7|         21.54|            150.78|
|      2|2024-06-28|      1048|            3|         33.87|101.60999999999999|
|      3|2024-04-02|      1065|            4|         85.17|            340.68|
|      4|2024-01-15|      1035|            3|         39.44|            118.32|
|      5|2024-04-16|      1053|            1|         53.93|             53.93|
|      6|2024-03-12|      1025|            8|          94.7|             757.6|
|      7|2024-01-21|      1074|            2|         30.44|             60.88|
|      8|2024-04-12|      1071|            1|         45.07|             45.07|
|      9|2024-05-01|      1039|            8|         64.58|            516.64|
|     10|2024-03-15|      1042|         